In [1]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f
findspark.init()

# Creating a SparkSession in Python
spark = SparkSession.builder\
    .master("local")\
    .appName("Spark Streaming Task 1")\
    .getOrCreate()

# # Configures the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.shuffle.partitions", "10")

your 131072x1 screen size is bogus. expect trouble
24/05/16 20:37:55 WARN Utils: Your hostname, PhDDevice resolves to a loopback address: 127.0.1.1; using 172.24.86.99 instead (on interface eth0)
24/05/16 20:37:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/16 20:37:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import os
cwd = os.getcwd()
cwd

'/home/dat_21127240/BigData/Big-Data-With-Seaborn-Bokeh-Plotly/Lab_03/src'

In [3]:
inputPath = cwd + "/../data/taxi-data"
outputPath = cwd + "/../output_2"
checkpointPath = cwd + "/../checkpoint"

In [4]:
full_schema = StructType([StructField(f"_c{i}", StringType(), nullable=True) for i in range(22)])

# define schema for yellow taxi trips
yellow_taxi_schema = StructType([
    StructField("type", StringType(), nullable=False),
    StructField("VendorID", IntegerType()),
    StructField("tpep_pickup_datetime", TimestampType()),
    StructField("tpep_dropoff_datetime", TimestampType()),
    StructField("passenger_count", IntegerType()),
    StructField("trip_distance", FloatType()),
    StructField("pickup_longitude", FloatType()),
    StructField("pickup_latitude", FloatType()),
    StructField("RatecodeID", FloatType()),
    StructField("store_and_fwd_flag", StringType()),
    StructField("dropoff_longitude", FloatType()),
    StructField("dropoff_latitude", FloatType()),
    StructField("payment_type", IntegerType()),
    StructField("fare_amount", FloatType()),
    StructField("extra", FloatType()),
    StructField("mta_tax", FloatType()),
    StructField("tip_amount", FloatType()),
    StructField("tolls_amount", FloatType()),
    StructField("improvement_surcharge", FloatType()),
    StructField("total_amount", FloatType())
])
# define schema for green taxi trips
green_taxi_schema = StructType([
    StructField("type", StringType(), nullable=False),
    StructField("VendorID", IntegerType()),
    StructField("lpep_pickup_datetime", TimestampType()),
    StructField("Lpep_dropoff_datetime", TimestampType()),
    StructField("Store_and_fwd_flag", StringType()),
    StructField("RateCodeID", IntegerType()),
    StructField("Pickup_longitude", FloatType()),
    StructField("Pickup_latitude", FloatType()),
    StructField("Dropoff_longitude", FloatType()),
    StructField("Dropoff_latitude", FloatType()),
    StructField("Passenger_count", IntegerType()),
    StructField("Trip_distance", FloatType()),
    StructField("Fare_amount", FloatType()),
    StructField("Extra", FloatType()),
    StructField("MTA_tax", FloatType()),
    StructField("Tip_amount", FloatType()),
    StructField("Tolls_amount", FloatType()),
    StructField("Ehail_fee", FloatType()),
    StructField("improvement_surcharge", FloatType()),
    StructField("Total_amount", FloatType()),
    StructField("Payment_type", IntegerType()),
    StructField("Trip_type", IntegerType())
])

In [5]:
# read the entire data files first
full_df = spark.readStream\
    .option("maxFilesPerTrigger", 100)\
    .csv(f"file://{inputPath}",
         header=False,
         schema=full_schema)

# Filter rows based on the 'type' column
yellow_df = full_df.where(full_df["_c0"] == "yellow").selectExpr(
    *[f"cast(_c{i} as {field.dataType.simpleString()}) as {field.name}" \
        for i, field in enumerate(yellow_taxi_schema.fields)]
)

green_df = full_df.where(full_df["_c0"] == "green").selectExpr(
    *[f"cast(_c{i} as {field.dataType.simpleString()}) as {field.name}" \
        for i, field in enumerate(green_taxi_schema.fields)]
)

In [6]:
sub_yellow = yellow_df.selectExpr("type")
sub_green = green_df.selectExpr("type")

In [7]:
sub_df = sub_yellow.union(sub_green)

In [8]:
by_type = sub_df.groupBy("type")\
    .count()

In [9]:
query = by_type.writeStream\
    .format("console")\
    .outputMode("complete")\
    .queryName("Type_Count")\
    .start()
query.awaitTermination(10)

24/05/16 20:39:35 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2f9763ac-95bd-4ae7-bf63-ccaf863551ef. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/05/16 20:39:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-----+
|  type|count|
+------+-----+
|yellow|17170|
| green| 2011|
+------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|  type|count|
+------+-----+
|yellow|36047|
| green| 4175|
+------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|  type|count|
+------+-----+
|yellow|55447|
| green| 6344|
+------+-----+



False

-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|  type|count|
+------+-----+
|yellow|75691|
| green| 8767|
+------+-----+

-------------------------------------------
Batch: 4
-------------------------------------------
+------+-----+
|  type|count|
+------+-----+
|yellow|94317|
| green|10949|
+------+-----+

-------------------------------------------
Batch: 5
-------------------------------------------
+------+------+
|  type| count|
+------+------+
|yellow|113752|
| green| 13207|
+------+------+

-------------------------------------------
Batch: 6
-------------------------------------------
+------+------+
|  type| count|
+------+------+
|yellow|133414|
| green| 15560|
+------+------+

-------------------------------------------
Batch: 7
-------------------------------------------
+------+------+
|  type| count|
+------+------+
|yellow|153316|
| green| 17926|
+------+------+



-------------------------------------------
Batch: 8
-------------------------------------------
+------+------+
|  type| count|
+------+------+
|yellow|186547|
| green| 22509|
+------+------+

-------------------------------------------
Batch: 9
-------------------------------------------
+------+------+
|  type| count|
+------+------+
|yellow|220773|
| green| 26970|
+------+------+



-------------------------------------------
Batch: 10
-------------------------------------------
+------+------+
|  type| count|
+------+------+
|yellow|254973|
| green| 31505|
+------+------+

-------------------------------------------
Batch: 11
-------------------------------------------
+------+------+
|  type| count|
+------+------+
|yellow|289111|
| green| 36074|
+------+------+

-------------------------------------------
Batch: 12
-------------------------------------------
+------+------+
|  type| count|
+------+------+
|yellow|323586|
| green| 40529|
+------+------+



-------------------------------------------
Batch: 13
-------------------------------------------
+------+------+
|  type| count|
+------+------+
|yellow|357839|
| green| 45001|
+------+------+

-------------------------------------------
Batch: 14
-------------------------------------------
+------+------+
|  type| count|
+------+------+
|yellow|370999|
| green| 46741|
+------+------+



In [10]:
query.stop()
spark.stop()